In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from prompts import gen_itinerary_prompt
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from utils import input_samples

def get_llm():
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, max_tokens=1000)
    return llm

def get_embeddings():
    embeddings = OpenAIEmbeddings()
    return embeddings

llm = get_llm()

gen_itinerary_chain = gen_itinerary_prompt | llm | StrOutputParser()

In [3]:
input_samples[0]

{'city': 'Hanoi',
 'trip_type': 'solo trip',
 'with_pets': 'No',
 'interests': ['must-see attraction',
  'cultural landmarks',
  'great food',
  'hidden gems',
  'shopping'],
 'other_interests': 'Photography',
 'duration': '4 days',
 'start_date': '01/12/2025',
 'end_date': '01/16/2025'}

In [4]:
response = gen_itinerary_chain.invoke(input_samples[0])

In [5]:
print(response)

```json
[
    {
        "day": 1,
        "date": "01/12/2025",
        "activities": [
            ["Hoan Kiem Lake", "Arrive in Hanoi and take a leisurely stroll around Hoan Kiem Lake, capturing beautiful photos of the scenery."],
            ["Ngoc Son Temple", "Visit Ngoc Son Temple located on an island in the lake, a must-see cultural landmark."],
            ["Old Quarter", "Explore the Old Quarter, known for its narrow streets, vibrant atmosphere, and unique shops."],
            ["Street Food Tour", "Join a street food tour in the evening to taste local delicacies like Pho and Bun Cha."]
        ]
    },
    {
        "day": 2,
        "date": "01/13/2025",
        "activities": [
            ["Temple of Literature", "Visit the Temple of Literature, Vietnam's first university, and take photos of its stunning architecture."],
            ["Vietnam Museum of Ethnology", "Explore the Vietnam Museum of Ethnology to learn about the diverse cultures of Vietnam."],
            ["Lunch

In [29]:
def extract_itinerary(response: str):
    import json
    import re

    match = re.search(r'```json(.*?)```', response, re.DOTALL)
    if match:
        itinerary_str = match.group(1).strip()
        # print(itinerary_str)

    itinerary_json = json.loads(itinerary_str)
    # return {"itinerary": itinerary_str}
    return itinerary_json

In [33]:
itinerary = extract_itinerary(response)
itinerary

[{'day': 1,
  'date': '01/12/2025',
  'activities': [['Hoan Kiem Lake',
    'Arrive in Hanoi and take a leisurely stroll around Hoan Kiem Lake, capturing beautiful photos of the scenery.'],
   ['Ngoc Son Temple',
    'Visit Ngoc Son Temple located on an island in the lake, a must-see cultural landmark.'],
   ['Old Quarter',
    'Explore the Old Quarter, known for its narrow streets, vibrant atmosphere, and unique shops.'],
   ['Street Food Tour',
    'Join a street food tour in the evening to taste local delicacies like Pho and Bun Cha.']]},
 {'day': 2,
  'date': '01/13/2025',
  'activities': [['Temple of Literature',
    "Visit the Temple of Literature, Vietnam's first university, and take photos of its stunning architecture."],
   ['Vietnam Museum of Ethnology',
    'Explore the Vietnam Museum of Ethnology to learn about the diverse cultures of Vietnam.'],
   ['Lunch at Cha Ca Thang Long',
    'Enjoy lunch at Cha Ca Thang Long, famous for its grilled fish.'],
   ['Shopping at Dong Xu

In [37]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator
# from langchain.output_parsers import PydanticOutputParser
from typing import List

class Activity(BaseModel):
    place: str
    activity: str

class DayPlan(BaseModel):
    day: int
    date: str
    activities: List[Activity]

class TripPlan(BaseModel):
    itinerary: List[DayPlan]

# parser = PydanticOutputParser(pydantic_object=TripPlan)

# chain = gen_itinerary_prompt | llm | StrOutputParser() | extract_itinerary_str | parser

# chain.invoke(input_samples[0])

In [39]:
from typing import List, Dict

def convert_to_trip_plan(raw_data: List[Dict]) -> TripPlan:
    """
    Converts raw trip data into a TripPlan Pydantic model.

    Args:
        raw_data (List[Dict]): The raw data containing day plans and activities.

    Returns:
        TripPlan: A Pydantic model representing the trip plan.
    """
    return TripPlan(
        itinerary=[
            DayPlan(
                day=day['day'],
                date=day['date'],
                activities=[
                    Activity(place=activity[0], activity=activity[1]) for activity in day['activities']
                ]
            )
            for day in raw_data
        ]
    )

# Example usage
trip_plan = convert_to_trip_plan(itinerary)
print(trip_plan)


itinerary=[DayPlan(day=1, date='01/12/2025', activities=[Activity(place='Hoan Kiem Lake', activity='Arrive in Hanoi and take a leisurely stroll around Hoan Kiem Lake, capturing beautiful photos of the scenery.'), Activity(place='Ngoc Son Temple', activity='Visit Ngoc Son Temple located on an island in the lake, a must-see cultural landmark.'), Activity(place='Old Quarter', activity='Explore the Old Quarter, known for its narrow streets, vibrant atmosphere, and unique shops.'), Activity(place='Street Food Tour', activity='Join a street food tour in the evening to taste local delicacies like Pho and Bun Cha.')]), DayPlan(day=2, date='01/13/2025', activities=[Activity(place='Temple of Literature', activity="Visit the Temple of Literature, Vietnam's first university, and take photos of its stunning architecture."), Activity(place='Vietnam Museum of Ethnology', activity='Explore the Vietnam Museum of Ethnology to learn about the diverse cultures of Vietnam.'), Activity(place='Lunch at Cha C

In [40]:
trip_plan.itinerary

[DayPlan(day=1, date='01/12/2025', activities=[Activity(place='Hoan Kiem Lake', activity='Arrive in Hanoi and take a leisurely stroll around Hoan Kiem Lake, capturing beautiful photos of the scenery.'), Activity(place='Ngoc Son Temple', activity='Visit Ngoc Son Temple located on an island in the lake, a must-see cultural landmark.'), Activity(place='Old Quarter', activity='Explore the Old Quarter, known for its narrow streets, vibrant atmosphere, and unique shops.'), Activity(place='Street Food Tour', activity='Join a street food tour in the evening to taste local delicacies like Pho and Bun Cha.')]),
 DayPlan(day=2, date='01/13/2025', activities=[Activity(place='Temple of Literature', activity="Visit the Temple of Literature, Vietnam's first university, and take photos of its stunning architecture."), Activity(place='Vietnam Museum of Ethnology', activity='Explore the Vietnam Museum of Ethnology to learn about the diverse cultures of Vietnam.'), Activity(place='Lunch at Cha Ca Thang L

In [41]:
# gen_itinerary_prompt | llm | StrOutputParser() | extract_itinerary | convert_to_trip_plan

In [28]:
import requests

In [21]:
from schemas import UserInput, TripPlan

In [22]:
user_input = UserInput(
    destination='Hanoi',
    trip_type='solo trip',
    with_pets=False,
    interests=['must-see attraction', 'cultural landmarks', 'great food', 'hidden gems', 'shopping'],
    other_interests='Photography',
    duration=4,
    start_date='01/12/2025',
    end_date='01/16/2025'
)
user_input


UserInput(destination='Hanoi', trip_type='solo trip', with_pets=False, interests=['must-see attraction', 'cultural landmarks', 'great food', 'hidden gems', 'shopping'], other_interests='Photography', duration=4, start_date='01/12/2025', end_date='01/16/2025')

In [23]:
import json

In [24]:
user_input.dict()

{'destination': 'Hanoi',
 'trip_type': 'solo trip',
 'with_pets': False,
 'interests': ['must-see attraction',
  'cultural landmarks',
  'great food',
  'hidden gems',
  'shopping'],
 'other_interests': 'Photography',
 'duration': 4,
 'start_date': '01/12/2025',
 'end_date': '01/16/2025'}

In [38]:

def send_requests(user_input: UserInput):
    # response = requests.post("http://127.0.0.1:8000/generate-itinerary/", params={"user_input": user_input})
    response = requests.post("http://127.0.0.1:8000/generate-itinerary/", data=user_input.json())
    return response.json() if response.status_code == 200 else {"status": "Error", "message": "Backend not reachable"}

In [39]:
response = send_requests(user_input)
response

{'itinerary': [{'day': 1,
   'date': '01/12/2025',
   'activities': [{'place': 'Hoan Kiem Lake',
     'activity': 'Arrive in Hanoi and take a leisurely stroll around Hoan Kiem Lake, a must-see attraction.'},
    {'place': 'Ngoc Son Temple',
     'activity': 'Visit Ngoc Son Temple located on an island in the lake.'},
    {'place': 'Old Quarter',
     'activity': 'Explore the Old Quarter for shopping and hidden gems.'},
    {'place': 'Street Food Tour',
     'activity': 'Join a street food tour in the evening to taste local delicacies.'}]},
  {'day': 2,
   'date': '01/13/2025',
   'activities': [{'place': 'Ho Chi Minh Mausoleum',
     'activity': "Visit the Ho Chi Minh Mausoleum and learn about Vietnam's history."},
    {'place': 'One Pillar Pagoda',
     'activity': 'Explore the One Pillar Pagoda, a cultural landmark.'},
    {'place': 'Vietnamese Coffee',
     'activity': 'Enjoy a traditional Vietnamese coffee at a local café.'},
    {'place': 'Dong Xuan Market',
     'activity': 'Shop 

In [40]:
len(response['itinerary'])

4

In [36]:
response.json()

{'itinerary': [{'day': 1,
   'date': '01/12/2025',
   'activities': [{'place': 'Hoan Kiem Lake',
     'activity': 'Arrive in Hanoi and take a leisurely stroll around Hoan Kiem Lake, a must-see attraction.'},
    {'place': 'Ngoc Son Temple',
     'activity': 'Visit Ngoc Son Temple located on an island in the lake.'},
    {'place': 'Old Quarter',
     'activity': 'Explore the vibrant streets of the Old Quarter, perfect for photography and shopping.'},
    {'place': 'Street Food Tour',
     'activity': 'Join a street food tour in the evening to sample local delicacies like Pho and Bun Cha.'}]},
  {'day': 2,
   'date': '01/13/2025',
   'activities': [{'place': 'Temple of Literature',
     'activity': "Visit the Temple of Literature, Vietnam's first university and a cultural landmark."},
    {'place': 'Vietnam Museum of Ethnology',
     'activity': 'Explore the Vietnam Museum of Ethnology to learn about the diverse cultures of Vietnam.'},
    {'place': 'Lunch at Cha Ca La Vong',
     'activ

In [19]:
response = requests.post("http://127.0.0.1:8000/test-post/", params={"message": "Hello"})
response.json()

NameError: name 'requests' is not defined

In [1]:
from chains import get_itinerary, get_gen_itinerary_chain
from utils import input_samples


In [2]:
# get_gen_itinerary_chain().invoke(input_samples[0])

In [3]:
input_samples[0]

{'destination': 'Hanoi',
 'trip_type': 'solo trip',
 'with_pets': 'No',
 'interests': ['must-see attraction',
  'cultural landmarks',
  'great food',
  'hidden gems',
  'shopping'],
 'other_interests': 'Photography',
 'duration': '4 days',
 'start_date': '01/12/2025',
 'end_date': '01/16/2025'}

In [5]:
trip_plan = get_itinerary(input_samples[0])

In [18]:
trip_plan.dict()

{'itinerary': [{'day': 1,
   'date': '01/12/2025',
   'activities': [{'place': 'Hoan Kiem Lake',
     'activity': 'Arrive in Hanoi and take a leisurely stroll around Hoan Kiem Lake, a must-see attraction.'},
    {'place': 'Ngoc Son Temple',
     'activity': 'Visit Ngoc Son Temple located on an island in the lake.'},
    {'place': 'Old Quarter',
     'activity': 'Explore the Old Quarter for shopping and hidden gems.'},
    {'place': 'Street Food Tour',
     'activity': 'Join a street food tour to taste local delicacies like Pho and Bun Cha.'},
    {'place': 'Photography',
     'activity': 'Capture the vibrant street life and architecture of the Old Quarter.'}]},
  {'day': 2,
   'date': '01/13/2025',
   'activities': [{'place': 'Temple of Literature',
     'activity': "Visit the Temple of Literature, a cultural landmark and Vietnam's first university."},
    {'place': 'Vietnam Fine Arts Museum',
     'activity': 'Explore the Vietnam Fine Arts Museum to appreciate local art.'},
    {'plac

In [43]:
import datetime

# Example date object
date_object = datetime.date(2024, 11, 21)

# Get day of the week
day_of_week = date_object.strftime("%A")
print(day_of_week)


Thursday


In [42]:
date_object

datetime.datetime(2023, 11, 21, 0, 0)